<a href="https://colab.research.google.com/github/KKKKKIKKKK/BaekJoon.java/blob/master/%EA%B3%BC%EC%A0%9C_A_Multi_head_Attention%EC%9C%BC%EB%A1%9C_%EA%B0%90%EC%A0%95_%EB%B6%84%EC%84%9D_%EB%AA%A8%EB%8D%B8_%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer 실습

감정 분석 task에 RNN 대신 Transformer를 구현 및 적용

In [1]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00


In [3]:
pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 17.2 MB/s eta 0:00:00


In [4]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


ds = load_dataset("stanfordnlp/imdb")
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')


def collate_fn(batch):
  max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids)
  labels = torch.LongTensor(labels)

  return texts, labels


train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Self-attention



In [ ]:
from torch import nn
from math import sqrt


class SelfAttention(nn.Module):
    def __init__(self, input_dim, d_model):
        super().__init__()

        # 입력 차원 (input_dim): 입력 특성의 차원
        # d_model: 모델의 차원 (자기-어텐션에서 내부 표현의 크기)

        self.input_dim = input_dim
        self.d_model = d_model

        # 입력을 query (Q), key (K), value (V) 벡터로 투영하기 위한 선형 레이어들
        self.wq = nn.Linear(input_dim, d_model)  # Query를 위한 투영
        self.wk = nn.Linear(input_dim, d_model)  # Key를 위한 투영
        self.wv = nn.Linear(input_dim, d_model)  # Value를 위한 투영
        self.dense = nn.Linear(d_model, d_model) # 어텐션 후 출력 투영

        # key-query 점수에 대한 어텐션 가중치를 계산하기 위한 소프트맥스
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask):
        # x: 입력 텐서 (batch_size, seq_len, input_dim)
        # mask: 특정 위치(예: 패딩 토큰)를 어텐션하지 않도록 하는 마스크

        # 1단계: 입력 x에 대해 선형 투영을 적용하여 Q, K, V 생성
        q, k, v = self.wq(x), self.wk(x), self.wv(x)  # (batch_size, seq_len, d_model)

        # 2단계: 어텐션 점수 계산
        # q * k^T: query와 key 간의 유사도를 계산
        # 점수(score)의 최종 형태: (batch_size, seq_len, seq_len)
        score = torch.matmul(q, k.transpose(-1, -2))  # (B, S, D) * (B, D, S) = (B, S, S)

        # 3단계: d_model의 제곱근으로 점수를 스케일링하여 큰 값으로 인해 softmax에서 발생할 수 있는 불안정성 방지
        score = score / sqrt(self.d_model)

        # 4단계: 마스크가 제공되었을 경우, 특정 위치에 어텐션을 주지 않도록 점수에 마스크 적용
        # 마스크는 특정 위치에 매우 큰 음수를 추가하여 softmax에서 해당 위치의 확률을 거의 0에 가깝게 만듦
        if mask is not None:
            score = score + (mask * -1e9)  # 마스크가 1인 위치에 매우 큰 음수값 추가

        # 5단계: 어텐션 점수에 softmax 적용하여 어텐션 가중치 계산
        # 각 행이 1로 합산되어 각 위치에 대한 어텐션 확률을 나타냄
        score = self.softmax(score)  # (batch_size, seq_len, seq_len)

        # 6단계: 어텐션 가중치를 사용하여 value 벡터의 가중합 계산
        result = torch.matmul(score, v)  # (batch_size, seq_len, d_model)

        # 7단계: 최종 선형 투영 레이어를 통해 출력
        result = self.dense(result)  # (batch_size, seq_len, d_model)

        # 자기-어텐션의 최종 결과 반환
        return result


대부분은 Transformer 챕터에서 배운 수식들을 그대로 구현한 것에 불과합니다.
차이점은 `mask`의 존재여부입니다.
이전 챕터에서 우리는 가변적인 text data들에 padding token을 붙여 하나의 matrix로 만든 방법을 배웠습니다.
실제 attention 계산에서는 이를 무시해주기 위해 mask를 만들어 제공해주게 됩니다.
여기서 mask의 shape은 (B, S, 1)로, 만약 `mask[i, j] = True`이면 그 변수는 padding token에 해당한다는 뜻입니다.
이러한 값들을 무시해주는 방법은 shape이 (B, S, S)인 `score`가 있을 때(수업에서 배운 $A$와 동일) `score[i, j]`에 아주 작은 값을 더해주면 됩니다. 아주 작은 값은 예를 들어 `-1000..00 = -1e9` 같은 것이 있습니다.
이렇게 작은 값을 더해주고 나면 softmax를 거쳤을 때 0에 가까워지기 때문에 weighted sum 과정에서 padding token에 해당하는 `v` 값들을 무시할 수 있게 됩니다.

다음은 self-attention과 feed-forward layer를 구현한 모습입니다.

In [ ]:
class TransformerLayer(nn.Module):
    def __init__(self, input_dim, d_model, dff):
        super().__init__()

        # input_dim: 입력 특성의 차원
        # d_model: 모델의 차원 (어텐션에서 사용되는 내부 표현의 크기)
        # dff: 피드포워드 네트워크에서의 중간 차원 크기

        self.input_dim = input_dim
        self.d_model = d_model
        self.dff = dff

        # Self-Attention 모듈
        self.sa = SelfAttention(input_dim, d_model)  # 자기-어텐션

        # Feed-Forward Network (FFN)
        # 두 개의 선형 레이어와 ReLU 활성화 함수로 구성
        self.ffn = nn.Sequential(
            nn.Linear(d_model, dff),  # 첫 번째 선형 레이어 (d_model -> dff)
            nn.ReLU(),                # ReLU 활성화 함수
            nn.Linear(dff, d_model)   # 두 번째 선형 레이어 (dff -> d_model)
        )

    def forward(self, x, mask):
        # x: 입력 텐서 (batch_size, seq_len, input_dim)
        # mask: 어텐션에서 특정 위치를 가리기 위한 마스크

        # 1단계: Self-Attention 적용
        x = self.sa(x, mask)  # (batch_size, seq_len, d_model)

        # 2단계: 피드포워드 네트워크 적용
        x = self.ffn(x)  # (batch_size, seq_len, d_model)

        # 최종 출력 반환
        return x


보시다시피 self-attention의 구현이 어렵지, Transformer layer 하나 구현하는 것은 수업 때 다룬 그림과 크게 구분되지 않는다는 점을 알 수 있습니다.

## Positional encoding

이번에는 positional encoding을 구현합니다. Positional encoding의 식은 다음과 같습니다:
$$
\begin{align*} PE_{pos, 2i} &= \sin\left( \frac{pos}{10000^{2i/D}} \right), \\ PE_{pos, 2i+1} &= \cos\left( \frac{pos}{10000^{2i/D}} \right).\end{align*}
$$

이를 Numpy로 구현하여 PyTorch tensor로 변환한 모습은 다음과 같습니다:

In [5]:
import numpy as np


def get_angles(pos, i, d_model):
    # pos: 위치 (position, 즉 문장의 각 단어의 위치)
    # i: 차원 인덱스 (d_model 내 각 차원에 해당하는 인덱스)
    # d_model: 모델의 차원 수
    # 각 위치에 대해 특정 차원의 각도(rate)를 계산하는 함수
    # 각도(rate)는 10000의 거듭제곱으로 나눈 값으로 계산
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    # position: 최대 문장 길이 (입력 문장의 길이를 의미)
    # d_model: 모델의 차원 수 (어텐션과 관련된 내부 표현 크기)

    # 각 위치와 차원에 해당하는 각도를 계산 (각 위치에 대해 모든 차원에 대한 각도 값 생성)
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)

    # 짝수 인덱스 (0, 2, 4, ...)에 대해 sin 함수 적용
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # 홀수 인덱스 (1, 3, 5, ...)에 대해 cos 함수 적용
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    # 각 위치에 대한 위치 인코딩 값 (배치 차원을 추가)
    pos_encoding = angle_rads[None, ...]

    # 위치 인코딩을 torch.FloatTensor로 변환하여 반환
    return torch.FloatTensor(pos_encoding)


max_len = 400  # 최대 문장 길이 설정
print(positional_encoding(max_len, 256).shape)  # 256 차원으로 위치 인코딩을 수행한 결과의 shape 출력


torch.Size([1, 400, 256])


Positional encoding은 `angle_rads`를 구현하는 과정에서 모두 구현이 되었습니다. 여기서 `angle_rads`의 shape은 (S, D)입니다.
우리는 일반적으로 batch로 주어지는 shape이 (B, S, D)인 tensor를 다루기 때문에 마지막에 None을 활용하여 shape을 (1, S, D)로 바꿔주게됩니다.

위에서 구현한 `TransformerLayer`와 positional encoding을 모두 합친 모습은 다음과 같습니다:

In [13]:
import torch
from torch import nn
import numpy as np
from math import sqrt


# SelfAttention 클래스 정의
class SelfAttention(nn.Module):
    def __init__(self, input_dim, d_model):
        super().__init__()

        self.input_dim = input_dim
        self.d_model = d_model

        self.wq = nn.Linear(input_dim, d_model)
        self.wk = nn.Linear(input_dim, d_model)
        self.wv = nn.Linear(input_dim, d_model)
        self.dense = nn.Linear(d_model, d_model)

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask):
        q, k, v = self.wq(x), self.wk(x), self.wv(x)
        score = torch.matmul(q, k.transpose(-1, -2))
        score = score / sqrt(self.d_model)

        if mask is not None:
            score = score + (mask * -1e9)

        score = self.softmax(score)
        result = torch.matmul(score, v)
        result = self.dense(result)

        return result


# TransformerLayer 클래스 정의
class TransformerLayer(nn.Module):
    def __init__(self, input_dim, d_model, dff):
        super().__init__()

        self.input_dim = input_dim
        self.d_model = d_model
        self.dff = dff

        self.sa = SelfAttention(input_dim, d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, dff),
            nn.ReLU(),
            nn.Linear(dff, d_model)
        )

    def forward(self, x, mask):
        x = self.sa(x, mask)
        x = self.ffn(x)

        return x


# 위치 인코딩 함수 정의
def positional_encoding(position, d_model):
    def get_angles(pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        return pos * angle_rates

    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])  # 짝수 인덱스에 대해 sin 적용
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])  # 홀수 인덱스에 대해 cos 적용
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


# TextClassifier 클래스 정의
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, d_model, n_layers, dff):
        super().__init__()

        self.vocab_size = vocab_size
        self.d_model = d_model
        self.n_layers = n_layers
        self.dff = dff

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = nn.parameter.Parameter(positional_encoding(400, d_model), requires_grad=False)
        self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff) for _ in range(n_layers)])
        self.classification = nn.Linear(d_model, 1)

    def forward(self, x):
        mask = (x == tokenizer.pad_token_id)
        mask = mask[:, None, :]
        seq_len = x.shape[1]

        x = self.embedding(x)
        x = x * sqrt(self.d_model)
        x = x + self.pos_encoding[:, :seq_len]

        for layer in self.layers:
            x = layer(x, mask)

        x = x[:, 0]
        x = self.classification(x)

        return x


# 모델 인스턴스 생성: 어휘 크기, 모델 차원, 레이어 수, FFN 차원을 지정하여 생성
model = TextClassifier(len(tokenizer), 32, 2, 32)



## 학습

학습하는 코드는 기존 실습들과 동일하기 때문에 마지막 결과만 살펴보도록 하겠습니다.

In [15]:
from torch.optim import Adam

# 학습률 설정 (learning rate)
lr = 0.001

# 모델을 GPU로 이동 ('cuda' 장치로 모델을 옮겨 GPU에서 연산 가능하게 설정)
model = model.to('cuda')

# 손실 함수 정의: BCEWithLogitsLoss는 이진 분류에서 사용되는 손실 함수
# 모델의 출력이 로지츠(logits)일 때 사용 (시그모이드 함수가 내부적으로 포함됨)
loss_fn = nn.BCEWithLogitsLoss()

# 옵티마이저 정의: Adam 옵티마이저 사용 (모델의 파라미터들을 업데이트할 때 사용)
# 학습률(lr)을 0.001로 설정
optimizer = Adam(model.parameters(), lr=lr)



In [16]:
import numpy as np
import matplotlib.pyplot as plt

# 모델의 정확도를 계산하는 함수
def accuracy(model, dataloader):
    cnt = 0  # 전체 샘플 수를 저장하는 변수
    acc = 0  # 맞춘 샘플 수를 저장하는 변수

    # 데이터 로더에서 배치 단위로 데이터를 가져옴
    for data in dataloader:
        inputs, labels = data  # 배치에서 입력(inputs)과 라벨(labels)을 가져옴
        inputs, labels = inputs.to('cuda'), labels.to('cuda')  # 입력과 라벨을 GPU로 이동

        # 모델을 통해 예측(preds)을 수행
        preds = model(inputs)
        # preds = torch.argmax(preds, dim=-1)  # 다중 클래스 분류의 경우 사용할 수 있는 코드 (주석 처리됨)

        # 이진 분류의 경우, 로지츠(logits)가 0보다 크면 1, 작으면 0으로 변환
        preds = (preds > 0).long()[..., 0]  # 예측값을 0 또는 1로 변환하고 차원을 맞춤

        # 배치 크기만큼 샘플 수를 더해줌
        cnt += labels.shape[0]

        # 예측값과 실제 라벨이 일치하는 개수를 더해줌
        acc += (labels == preds).sum().item()

    # 전체 샘플 중에서 맞춘 비율을 반환 (정확도)
    return acc / cnt


In [ ]:
n_epochs = 50

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda').float()

    preds = model(inputs)[..., 0]
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 223.68699997663498
=========> Train acc: 0.796 | Test acc: 0.767
Epoch   1 | Train Loss: 172.1974290907383
=========> Train acc: 0.834 | Test acc: 0.785
Epoch   2 | Train Loss: 149.595973610878
=========> Train acc: 0.862 | Test acc: 0.794
Epoch   3 | Train Loss: 130.4671006053686
=========> Train acc: 0.892 | Test acc: 0.802
Epoch   4 | Train Loss: 113.0657674074173
